<center><img src="http://i.imgur.com/sSaOozN.png" width="500"></center>

## Course: Computational Thinking for Governance Analytics

### Prof. José Manuel Magallanes, PhD 
* Visiting Professor of Computational Policy at Evans School of Public Policy and Governance, and eScience Institute Senior Data Science Fellow, University of Washington.
* Professor of Government and Political Methodology, Pontificia Universidad Católica del Perú. 

_____

# Data Preprocessing in Python: Data Integration and Reshaping

We all know collect data from different places. While the cleaning and formatting is done for each data source, we finally need to integrate all the sources into one to start the real analytical work.

I will use several data sets in this material. Let me start with the one from CIA on internet users:

In [ ]:
import pandas as pd

interlink="https://www.cia.gov/the-world-factbook/field/internet-users/country-comparison"

ciainter=pd.read_html(interlink)[0]

The object **ciainter** is a data frame. Notice the **[0]** at the end of the **pd.read_html()** function, you should remember that functions returns a **list** of elements. The data frame was the first one (position zero). Let's see some info:

In [ ]:
ciainter.info()

The dataframe **ciainter** has four columns and 228 rows. The column **Dtype** informs the data type of each column in the data frame. Two columns are numbers (integers) and two are text (object). The third column has a weird name, let's see the data:

In [ ]:
ciainter.head()

The unnamed column is the one informing the amount of internet users, then let's rename it:

In [ ]:
ciainter.rename(columns={'Unnamed: 2':'intusers'},inplace=True) #notice inplace which changes the data frame.

I will not use the rank nor the date of information, so I will delete (drop) both:

In [ ]:
bye = ['Rank','Date of Information'] 
ciainter.drop(columns=bye, inplace=True) 

Country is a **key** column. If I were to add more columns from other data frame, I will use this column for merging. I will make sure it has no trailing or leading spaces:

In [ ]:
ciainter['Country']=ciainter['Country'].str.strip()

The amount of users is influenced by the population size. So, it is better to make divide the users by the population. I need the population data, which the CIA also has:

In [ ]:
poplink='https://www.cia.gov/the-world-factbook/field/population/country-comparison'
ciapop=pd.read_html(poplink)[0]

Let's see what we got:

In [ ]:
ciapop.info()

In [ ]:
ciapop.head()

These data needs the same as the previous one:

In [ ]:
ciapop.rename(columns={'Unnamed: 2':'pob'},inplace=True)
ciapop['Country']=ciapop['Country'].str.strip()
ciapop.drop(columns=bye, inplace=True) 

## Merging

Integrating data sets needs the following considerations:

* Merging is done on two data frames (you can prepare a function to merge more).
* You need a common column to be used in both data frames. The column names can be different.
* The merge can keep only the full coincidences, or also the values not matched, which will help you detect possible extra cleaning.
* Pandas differentiates the **left** from the **right** data frames.

Since I want to divide the number of internet users by the population, I need to **merge** both data frames. Let me show you several possibilities:

* **Option one**: merge only the coincidences:

In [ ]:

ciainter.merge(ciapop)

The previous merge just got rid of any row that could not find the same country name in both data frames.

* **Option two**: merge when the column keys are different:

In [ ]:
#let me rename the key column in 'ciapop':
ciapop.rename(columns={'Country':'countries'},inplace=True)

In [ ]:
# this will give you an error:
ciainter.merge(ciapop)

In [ ]:
# this is the rigth code:
ciainter.merge(ciapop,left_on='Country',right_on='countries')

You got the same result (with an extra column).

* **Option three**: you want to keep all the rows in the **left** data frame:

In [ ]:
ciainter.merge(ciapop,left_on='Country',right_on='countries',how='left') 

* **Option four**: you want to keep all the rows in the **right** data frame:

In [ ]:
ciainter.merge(ciapop,left_on='Country',right_on='countries',how='right') 

* **Option five**: you want to keep all the rows from **both** data frames:

In [ ]:
ciainter.merge(ciapop,left_on='Country',right_on='countries',how='outer',indicator='True') 

Notice that I included the argument **indicator=True**, which added a column telling if the row comes from both, or from the left or rigth data frame.

### Looking for improvements after merging

Let me pay attention to this result again:

In [ ]:
allRight=ciainter.merge(ciapop,left_on='Country',right_on='countries',how='left', indicator=True) 
allRight

The previous result is different from this one:

In [ ]:
ciainter.merge(ciapop,left_on='Country',right_on='countries')

There is **one** row difference, let me see:

In [ ]:
allRight[allRight._merge!='both']

I have found the only country that is not present in 'ciapop'. Imagine you had **The Antarctica** in *ciapop*, you could replace it like this:

In [ ]:
###dictionary of replacements:
#replacementscia={'The Antarctica':'Antarctica'}

### replacing
#ciapop.countries.replace(replacementscia,inplace=True)

...and you will need to redo the merge.

Let me keep the **allRight** dataframe, erasing the irrelevant columns and rows:

In [ ]:
# dropping columns
byeCols=['countries','_merge']
allRight.drop(columns=byeCols,inplace=True)

In [ ]:
# dropping rows
byeRows=[217]
allRight.drop(index=byeRows,inplace=True)

In [ ]:
When you erase 

____
____


### <font color="red">Saving File to Disk</font>

#### For future use in Python:

In [ ]:
allRight.to_pickle("allRight.pkl")
# you will need: DF=pd.read_pickle("interhdi.pkl")
# or:
# from urllib.request import urlopen
# DF=pd.read_pickle(urlopen("https://..../interhdi.pkl"),compression=None)

#### For future  use in R:

In [ ]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr

base = importr('base')
base.saveRDS(allRight,file="allRight.RDS")

#In R, you call it with: DF = readRDS("interhdi.RDS")
#or, if iyou read from cloud: DF = readRDS(url("https://..../interhdi.RDS")

## RESHAPING

### Wide and Long format

The current format of **allRight** is known as the **WIDE** format. In this format, the variables are in every column, the most traditional one for spreadsheet users. Several functions are ready to use this format, for example:

In [ ]:
# A scatter plot
allRight.plot.scatter(x='intusers', y='pob',grid=True)

In [ ]:
# a boxplot
allRight.loc[:,['intusers','pob']].boxplot(vert=False,figsize=(15,5),grid=False)

However, the wide format may be less efficient for some packages:

In [ ]:
#!pip install plotnine

In [ ]:
import plotnine as p9

base=p9.ggplot(data=allRight)
base + p9.geom_boxplot(p9.aes(x=1,y='intusers')) + p9.geom_boxplot(p9.aes(x=2,y='pob'))

Let's see the **LONG** format:

In [ ]:
allRight.melt(id_vars=['Country'])

The amount of of rows multiplies, but **all** the variables in the wide format will use only **TWO** columns in the wide format (in its basic form). Notice the difference in this code:

In [ ]:
allRightLONG=allRight.melt(id_vars=['Country'])
base=p9.ggplot(data=allRightLONG)
base + p9.geom_boxplot(p9.aes(x='variable',y='value'))

### Transposing

We have two data sets on information about race, one for California and one for Washington State. These are the links:

In [ ]:
# California link
linkCa='https://github.com/EvansDataScience/data/raw/master/CaliforniaRace.xlsx'

# Washington link
linkWa='https://github.com/EvansDataScience/data/raw/master/WAraceinfo.xlsx'

You can realize from the links that both data are in Excel format ( _xlsx_ ). Let's fetch them:

In [ ]:
raceca=pd.read_excel(linkCa,0) # first sheet
racewa=pd.read_excel(linkWa,1) # second sheet

Let me see what **racewa** has:

In [ ]:
racewa

The rows give you information on geographical units (the **unit of analysis** is the county). It apparently starts with information of the whole state (Washington), and then county by county. Notice that units of analysis repeat by group age and by year.

Now, let's see what **raceca** has:

In [ ]:
raceca

Notice that the data from California speaks of the same, but the **units of analysis** (counties) appear in the columns. Notice that while WA State only shows counts, CA State also shows percentages. 

The data from WA State is a standard format for data frames, while the one in CA State is not. However, a simple operation known as **transposing** will solve the situation:

In [ ]:
raceca.transpose()

In [ ]:
# Let's make the changes:
raceca=raceca.transpose()

The transposed data frame requires several cleaning steps:

* Move first row as column names:

In [ ]:
# first row, where the columns names are.
raceca.columns=raceca.iloc[0,:].to_list()

* Delete first row:

In [ ]:
raceca.head()

In [ ]:
# dropping first row effective immediatly
raceca.drop(index='Unnamed: 0',inplace=True)

* Keep the columns about **race**:

In [ ]:
# finding positions:
list(enumerate(raceca.columns))

In [ ]:
# values needed:
[0]+ list(range(23,31))

In [ ]:
# keeping the ones I want:
raceca=raceca.iloc[:,[0]+ list(range(23,31))]
raceca

* Drop rows with missing values:

In [ ]:
raceca.dropna(subset=['Statistics'],inplace=True)

When we drop rows, we reset indexes:

In [ ]:
raceca.reset_index(drop=True,inplace=True)

In [ ]:
# currently
raceca

This is a much  simpler data frame. 

### Aggregating

The data from WA State has data from different years, while the one from CA is just from 2019. Let's keep that year for WA:

In [ ]:
racewa.query('Year==2019',inplace=True)

Now you have:

In [ ]:
racewa

Notice that the data is organized by age in WA:

In [ ]:
racewa['Age Group'].to_list()

There is a **Total** in **Age Group** that I will not use (that  makes this work simpler).

In [ ]:
racewa=racewa[racewa['Age Group']!='Total']
racewa

The ages are organized in intervals, let's keep the consecutive ones:

In [ ]:
stay=['0-19', '20-64', '65+']

racewa=racewa[racewa['Age Group'].isin(stay)]
racewa

* We should keep the values that do not include '__Washington__'

In [ ]:
racewa=racewa[racewa["Area Name"]!='Washington']
racewa

The **aggregation** is used when you need to colapse rows. You can use different function for collapsing, in this case we will **sum** within each county, so I can get a total per county:

In [ ]:
racewa=racewa.groupby(['Area Name','Area ID','Year']).sum()
racewa

The **Age Group** is not used in the aggreting function **groupby()**, but it is the only **non-numeric**  columns that is not used in this function. Notice that **Age Group** values have been concatenated, and the grouping variables are the **indexes** (row names).

You can drop the age group now:

In [ ]:
racewa.drop(columns=['Age Group'],inplace=True)
racewa

### Appending

The units of analysis in both data frames are the same kind (counties) but they data from one data frame will not be a column for the other. In this situation, you do not **merge**, you **append**.

The condition for appending is that both data frame have the same colum names. Let's see:

In [ ]:
raceca.columns

In [ ]:
racewa.columns

The column 'Some Other Race Alone' in **raceca** has no similar value in **racewa**. Let's drop it:

In [ ]:
raceca.drop(columns=['Some Other Race Alone'],inplace=True)

The columns in WA State have values for male and female. Since CA State do not have that, we have to get rid of those:

In [ ]:
# good names
[name for name in racewa.columns if 'Total' in name]

In [ ]:
# then
racewa=racewa.loc[:,[name for name in racewa.columns if 'Total' in name]]
racewa

We need to reciver the county names in racewa. They are part of the indexes. Let me use the **reset_index** funtion, but using the argument **drop=False**:

In [ ]:
racewa.reset_index(drop=False,inplace=True)

In [ ]:
# you have
racewa

The columns "Area ID" and "Year" are not present in raceca, we should drop them:

In [ ]:
racewa.drop(columns=["Area ID", "Year"],inplace=True)
racewa

Let's see the names of both:

In [ ]:
dict(zip(raceca.columns,racewa.columns))

You can use that dictionary to alter the names in raceca:
    

In [ ]:
raceca.rename(columns=dict(zip(raceca.columns,racewa.columns)),inplace=True)
raceca

The **raceca** has the word "California"; since you are combining data from different states, it is better you keep that info:

In [ ]:
# using 'expand'
raceca['Area Name'].str.split(pat=", ",expand=True)

The fucntion **str.split** creates two columns, let me save them here:

In [ ]:
twoCols=raceca['Area Name'].str.split(pat=", ",expand=True)
twoCols

Use the columns to replace other columns:

In [ ]:
raceca['Area Name']=twoCols[0]
raceca['State']=twoCols[1]

In [ ]:
# we have
raceca

Let's drop the las row (the 'TOTAL'):

In [ ]:
raceca.drop(index=[41],axis=0,inplace=True)

You can get rid of the 'County' string:

In [ ]:
raceca['Area Name']=raceca['Area Name'].str.replace(" County","")
raceca

The data frame  **racewa** does not have a "State" column, let me create it:

In [ ]:
racewa['State']='Washington'

Let's check the coincidences in  the column names:

In [ ]:
racewa.columns==raceca.columns

Now we can append:

In [ ]:
racewaca=racewa.append(raceca, ignore_index=True)
racewaca

Let's check the data types:

In [ ]:
#checking
racewaca.info()

There are several columns that are numeric, but they have the wrong Dtype. Let's solve this:

In [ ]:
# this require formatting
racewaca[racewaca.columns[1:-1]]

In [ ]:
# let's do it:

racewaca[racewaca.columns[1:-1]]=racewaca[racewaca.columns[1:-1]].astype('float')

In [ ]:
# checking
racewaca.info()